In [1]:
#!pip install -qq transformers genomic-benchmarks datasets

In [2]:
### Parameters
MODEL_NAME = "simecek/DNAMobileBert"
TOKENIZER_NAME = "armheb/DNA_bert_6"
K = 6
STRIDE = K

# if less than 1, only this fraction of each dataset is used
DATASET_THINING = 1

BENCHMARKS_FOLDER = '/root/.genomic_benchmarks'
# BENCHMARKS_FOLDER = '/home/jovyan/.genomic_benchmarks/' (for INFRA HUB)

DATASETS = [('demo_coding_vs_intergenomic_seqs', 0),
 ('demo_human_or_worm', 0), ('human_enhancers_cohn', 0), ('human_enhancers_ensembl', 0),
 ('human_ensembl_regulatory', 0), ('human_nontata_promoters', 0), ('human_ocr_ensembl', 0)]

BATCH_SIZE = 32
LEARNING_RATE = 8e-5
EPOCHS = 4
RUNS = 1

# do not forget to attach drive
OUTPUT_PATH = 'drive/MyDrive/genomic_benchmarks/DNAMobileBERT.csv'

## Download benchmark datasets and tokenizer

In [3]:
from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.data_check.info import is_downloaded
from pathlib import Path
from tqdm.autonotebook import tqdm

for dataset_name, dataset_version in tqdm(DATASETS):
    if not is_downloaded(dataset_name):
        download_dataset(dataset_name, version=dataset_version)

benchmark_root = Path(BENCHMARKS_FOLDER)

/usr/local/lib/python3.7/dist-packages/genomic_benchmarks/utils/datasets.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


  0%|          | 0/7 [00:00<?, ?it/s]

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME)

In [5]:
def kmers_strideK(s, k=K):
    return [s[i:i + k] for i in range(0, len(s), k) if i + k <= len(s)]

def kmers_stride1(s, k=K):
    return [s[i:i + k] for i in range(0, len(s)-k+1)]

if (STRIDE == 1):
  kmers = kmers_stride1
else:
  kmers = kmers_strideK

# function used for the actual tokenization
def tok_func(x): return tokenizer(" ".join(kmers(x["seq"])))

# example
tok_func({'seq': 'ATGGAAAGAGGCACCATTCT'})    

{'input_ids': [2, 501, 835, 650, 3], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}

## Looping through datasets, fine-tuning the model for each of them, logging metrics

In [6]:
import pandas as pd
import numpy as np
from random import random
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import Dataset, DatasetDict, load_metric

def compute_metrics_binary(eval_preds):
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

def compute_metrics_multi(eval_preds):
    metric = load_metric("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

outputs = []

for dataset_name, dataset_version in tqdm(DATASETS):
    

    labels = sorted([x.stem for x in (benchmark_root / dataset_name / 'train').iterdir()])

    tmp_dict = {}

    for split in ['train', 'test']:
        for nlabel, label in enumerate(labels):
            for f in (benchmark_root / dataset_name / split / label).glob('*.txt'):
                txt = f.read_text()
                if not DATASET_THINING or DATASET_THINING==1:
                    tmp_dict[f"{label} {f.stem}"] = (split, nlabel, txt)
                elif random() < DATASET_THINING:
                    tmp_dict[f"{label} {f.stem}"] = (split, nlabel, txt)

    df = pd.DataFrame.from_dict(tmp_dict).T.rename(columns = {0: "dset", 1: "cat", 2: "seq"})

    ds = Dataset.from_pandas(df)

    tok_ds = ds.map(tok_func, batched=False, remove_columns=['__index_level_0__', 'seq'])
    tok_ds = tok_ds.rename_columns({'cat':'labels'})

    dds = DatasetDict({
        'train': tok_ds.filter(lambda x: x["dset"] == "train").remove_columns('dset'),
        'test':  tok_ds.filter(lambda x: x["dset"] == "test").remove_columns('dset')
    })

    compute_metrics = compute_metrics_binary if len(labels) == 2 else compute_metrics_multi

    for _ in range(RUNS):

        model_cls = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(labels))

        args = TrainingArguments('outputs', learning_rate=LEARNING_RATE, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
            evaluation_strategy="epoch", per_device_train_batch_size=BATCH_SIZE, per_device_eval_batch_size=BATCH_SIZE*2,
            num_train_epochs=EPOCHS, weight_decay=0.01, save_steps=100000, report_to='none')
        
        trainer = Trainer(model_cls, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                          tokenizer=tokenizer, compute_metrics=compute_metrics)
        trainer.train()
        
        max_accuracy = max([x['eval_accuracy'] for x in trainer.state.log_history if 'eval_accuracy' in x])
        max_f1 = max([x['eval_f1'] for x in trainer.state.log_history if 'eval_f1' in x]) if len(labels) == 2 else np.nan
        train_runtime = max([x['train_runtime'] for x in trainer.state.log_history if 'train_runtime' in x])
        
        outputs.append((dataset_name, max_accuracy, max_f1, train_runtime))




  0%|          | 0/7 [00:00<?, ?it/s]

Parameter 'function'=<function tok_func at 0x7f617411bdd0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/100000 [00:00<?, ?ex/s]

  0%|          | 0/100 [00:00<?, ?ba/s]

  0%|          | 0/100 [00:00<?, ?ba/s]

...

  0%|          | 0/140 [00:00<?, ?ba/s]

  0%|          | 0/140 [00:00<?, ?ba/s]

loading configuration file https://huggingface.co/simecek/DNAMobileBert/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/56fbb8ccd0d0e99dd2b7a04de7110c5b777eb8a1ebb498361f7a1154688b2c59.99d23d2fc97afa64b897f7ece8789a51989ba39cf59363cf864da92dc476b524
Model config MobileBertConfig {
  "_name_or_path": "simecek/DNAMobileBert",
  "architectures": [
    "MobileBertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_activation": true,
  "classifier_dropout": null,
  "embedding_size": 128,
  "hidden_act": "relu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "intra_bottleneck_size": 128,
  "key_query_shared_bottleneck": true,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "mobilebert",
  "normalization_type": "no_norm",
  "num_attention_heads": 4,
  "num_feedforward_networks": 4,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "torch_dtype": "float32

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.000000,nan,0.500000,0.000000
2,0.000000,nan,0.500000,0.000000
3,0.000000,nan,0.500000,0.000000
4,0.000000,nan,0.500000,0.000000


***** Running Evaluation *****
  Num examples = 34952
  Batch size = 64
***** Running Evaluation *****
  Num examples = 34952
  Batch size = 64
***** Running Evaluation *****
  Num examples = 34952
  Batch size = 64
***** Running Evaluation *****
  Num examples = 34952
  Batch size = 64


Training completed. Do not forget to share your model on huggingface.co/models =)




## Outputs

In [7]:
outputs_df = pd.DataFrame(outputs, columns = ['dataset', 'accuracy', 'f1', 'train_runtime'])
outputs_df

,dataset,accuracy,f1,train_runtime
0,demo_coding_vs_intergenomic_seqs,0.500000,0.000000,2394.4881
1,demo_human_or_worm,0.500000,0.000000,2433.3550
2,human_enhancers_cohn,0.709269,0.728244,856.4418
3,human_enhancers_ensembl,0.789409,0.783968,3157.1434
4,human_ensembl_regulatory,0.370419,NaN,5621.5925
5,human_nontata_promoters,0.764999,0.789280,927.1230
6,human_ocr_ensembl,0.500000,0.000000,3662.0521


In [8]:
outputs_df.groupby('dataset').agg({'accuracy' : ['mean', 'sem'], 'f1' : ['mean','sem'], 'train_runtime': ['mean', 'sem']})

accuracy            f1     train_runtime    
                                      mean sem      mean sem          mean sem
dataset                                                                       
demo_coding_vs_intergenomic_seqs  0.500000 NaN  0.000000 NaN     2394.4881 NaN
demo_human_or_worm                0.500000 NaN  0.000000 NaN     2433.3550 NaN
human_enhancers_cohn              0.709269 NaN  0.728244 NaN      856.4418 NaN
human_enhancers_ensembl           0.789409 NaN  0.783968 NaN     3157.1434 NaN
human_ensembl_regulatory          0.370419 NaN       NaN NaN     5621.5925 NaN
human_nontata_promoters           0.764999 NaN  0.789280 NaN      927.1230 NaN
human_ocr_ensembl                 0.500000 NaN  0.000000 NaN     3662.0521 NaN

In [9]:
# saving outputs to csv file
outputs_df.to_csv(OUTPUT_PATH, index=False)